In [2]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
from sympy import *
%load_ext Cython
import time




T 108.33078115826872


In [3]:
%%cython

cdef extern from "math.h":
    double sin(double)
    const double pi "M_PI"
cdef float aep = 1.1560
cdef float app = 0.8669
cdef float m_p = 1836.2
cdef float E_0 = 0.6
cdef float w_0 = 0.058
cdef int N = 3
cdef float T = 2 * pi * N /w_0


cpdef  float length2(list R, list r):
    return pow((R[0] - r[0]),2)  + pow((R[1] - r[1]),2) + pow((R[2] - r[2]),2)
cpdef  float f_e(int i, list R_1, list R_2, list r):
    return (R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)), (3 / 2))) + (
                (R_2[i] - r[i])) /  (
                pow((length2(R_2,r) + pow(aep , 2)),(3 / 2)))
cpdef float f_p1(int i, list R_1, list R_2, list r) :
        return (-(R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)),  (3 / 2))) + (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))

cpdef float f_p2(int i, list R_1, list R_2, list r) :
        return (-(R_2[i] - r[i]) / (
                pow((length2(R_2,r) + pow(aep,2)) , (3 / 2))) - (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))
cpdef  list dif(  a, float t) :
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    dadt = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z])]
    return dadt
cpdef float E(  t):
    E = E_0 * pow((sin(pi * t / T)),2) * sin(w_0 * t)
    return E


cpdef list dif2(a, float t2):
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    L = 100000
    dadt  = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) - E(t2),
            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2), 
            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2)] 
    return dadt


In [7]:
p = mpar.p
R_1 = [-mpar.dist/2, 0, 0] #начальные положения pr1,pr2,e
R_2 = [mpar.dist/2, 0, 0]
r = [0,0,0]
#def   dif(  a,  t) :
#    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
#    dadt = [px, py, pz,
#            p_1x / mpar.m_p, p_1y / mpar.m_p, p_1z / mpar.m_p,
#            p_2x / mpar.m_p, p_2y / mpar.m_p, p_2z / mpar.m_p,
#            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z])]
#    return dadt
print (mpar.dist)
def a_0(i):
    a_0 = [0, 0, 0, -mpar.dist / 2, 0, 0, mpar.dist / 2, 0, 0, p[i][0], p[i][1], p[i][2], 0, 0, 0, 0, 0, 0]
    return a_0
solution = []
start = time.time()
print (odeint(dif, a_0(2), mpar.t))

#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))
#print ("rfn", solution[1][mpar.L -1,3])


#Решаем сиситему 18 диф. уравнений с лазером
#t = linspace(u, u+T,10000)

#def E(t):
#    E = mpar.E_0 * (np.sin(np.pi * t / mpar.T)) ** 2 * np.sin(mpar.w_0 * t)
#    return E

t2 = linspace(mpar.u, mpar.u + 325, 100000)


#def dif2(a, t2):
#    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
#    L = 100000
#    dadt  = [px, py, pz,
#            p_1x / mpar.m_p, p_1y / mpar.m_p, p_1z / mpar.m_p,
#            p_2x / mpar.m_p, p_2y / mpar.m_p, p_2z / mpar.m_p,
#            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) - E(t2),#-E(t - 300*mpar.T)
#            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2), #E(t - 300*mpar.T)
#            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
#            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2)] #E(t - 300*mpar.T)
#    return dadt

def a_00(i):
    a_00 = [solution[i][mpar.L - 1, 0], solution[i][mpar.L - 1, 1], solution[i][mpar.L - 1, 2], solution[i][mpar.L - 1, 3], solution[i][mpar.L - 1, 4], solution[i][mpar.L - 1, 5], solution[i][mpar.L - 1, 6], solution[i][mpar.L - 1, 7], solution[i][mpar.L - 1, 8], solution[i][mpar.L - 1, 9], solution[i][mpar.L - 1, 10], solution[i][mpar.L - 1, 11], solution[i][mpar.L - 1, 12], solution[i][mpar.L - 1, 13], solution[i][mpar.L - 1, 14], solution[i][mpar.L - 1, 15], solution[i][mpar.L - 1, 16], solution[i][mpar.L - 1, 17]]
    return a_00
solution2 = []

#print(solution[0][:, 0], solution[0][mpar.L-1, 0])

# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

1.5070016255625132
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.87241979e-04  6.42504436e-04 -1.03919965e-03 ...  5.33674900e-08
   4.06163077e-07 -6.56936360e-07]
 [ 1.57448329e-03  1.28500366e-03 -2.07839087e-03 ...  2.11935759e-07
   1.60013139e-06 -2.58808485e-06]
 ...
 [-7.62797944e-01  2.72563041e-01 -4.40848026e-01 ...  3.92196695e+00
   9.49183552e-02 -1.53522242e-01]
 [-7.63729932e-01  2.72171268e-01 -4.40214368e-01 ...  3.92199405e+00
   9.49408465e-02 -1.53558619e-01]
 [-7.64661341e-01  2.71778798e-01 -4.39579583e-01 ...  3.92202139e+00
   9.49632259e-02 -1.53594816e-01]]
Время выполнения: 25.917092084884644
Среднее время выполнения одного эксперимента: 2.5917092084884645
